In [ ]:
#rerun short scraper with modifications to capture headers

# Capstone Project Part 1: Data Collection

**Authur:** Kate Meredith  

**Date:** September-November 2022

**Notebook #:** 1 of

## Background

**Purpose:** The purpose of this notebook is to acquire the data to be analyzed.

**Source:** Data is collected from [CoffeeReview.com](https://www.coffeereview.com/).

## References
 
References used in scraping process:
- [Stack Overflow article](https://stackoverflow.com/questions/16627227/problem-http-error-403-in-python-3-web-scraping): to address issue with 403 error
- Referenced this [article](https://realpython.com/list-comprehension-python/) on using for loop to generate list.
- Help provided by Dan Drust (web developer) on reading HTML to select correct url links.
- Referred to [article](https://www.danielherediamejias.com/6-basic-tips-to-perform-web-scraping-with-python/) on using time sleep to avoid overloading server.
- Used [stackoverflow](https://stackoverflow.com/questions/31674557/how-to-append-rows-in-a-pandas-dataframe-in-a-for-loop) on appending code to pandas dataframe.
- To get past index errors, referenced Bruno's notebook and this [article](https://stackoverflow.com/questions/11902458/i-want-to-exception-handle-list-index-out-of-range)
- Referenced Pandas [documentation](https://pandas.pydata.org/docs/reference/api/pandas.concat.html) on combinined dataframes


In [ ]:
##### QUESTIONS FOR ARAD


In [ ]:
##### NEXT STEPS

#Include text file to reproduce environment (see environment and packages notebook)
#Try different approaches to modeling: 3 cleaned sets
    #remove rows with missing data
    #columns with missing data
    #impute missing data - can do this with modeling like KNN(?) not just mean to get better picture of data

In [1]:
#Importing libraries.
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import requests
from bs4 import BeautifulSoup as bs
import html5lib
from urllib.request import Request, urlopen

import time
import random

## Scraping URLs from all pages and appending to list

In [2]:
#this scrapes the url links for all the unique coffee review pages

url_list = [] #placeholder for url links

for page in range(1,342): #pages range 1-341
    req = Request(
        url=f'https://www.coffeereview.com/review/page/{page}/',  #indicates each page to pull links from
        headers={'User-Agent': 'Mozilla/5.0'} #gets around 403 error 
)
    page = urlopen(req).read()
    soup = bs(page, 'html.parser') #creating soup to search
    review_link = soup.find_all("p", class_="review-title") #finds all the title urls 

    for a in range(len(review_link)):
        url_list.append(review_link[a].find("a").get('href')) #turns urls into usable format

print(url_list) #prints url links
print(len(url_list)) #prints total number of url links

['https://www.coffeereview.com/review/colombia-jardines-del-eden-pink-bourbon-wine-yeast-honey/', 'https://www.coffeereview.com/review/kenya-aa-21/', 'https://www.coffeereview.com/review/ethiopia-bekele-heto-natural/', 'https://www.coffeereview.com/review/kenya-karindundu-ab/', 'https://www.coffeereview.com/review/kenya-ichuga-aa/', 'https://www.coffeereview.com/review/costa-rica-canet-raisin-honey/', 'https://www.coffeereview.com/review/birhanu-sisters-ethiopia/', 'https://www.coffeereview.com/review/bat-country-dark-roast-ethiopia/', 'https://www.coffeereview.com/review/kenya-gichithaini-aa/', 'https://www.coffeereview.com/review/colombia-cerro-azul-enano-3/', 'https://www.coffeereview.com/review/peru-incahuasi/', 'https://www.coffeereview.com/review/colombia-apontes-guardians/', 'https://www.coffeereview.com/review/nicaragua-flor-de-dalia-natural/', 'https://www.coffeereview.com/review/ethiopia-bench-maji-geisha-g1-natural/', 'https://www.coffeereview.com/review/reina-de-la-asuncion

## Background on Review Data Scraped from Each Page

**Notes about Scores**
 
Not all pages provice identical scoring metrics:
- Order scores are listed in differs
- Some contain scores on acidity, while others on "with milk"
- Some report fewer scores

Addressed by:
- Scraping both the header label and the score itself
- Header corresponds with score in column directly to the right of it
- In next phase, will clean data to sort out scores (making sure each column only captures one score type, determining how to handle nans)

**Notes about Paragraph Data**

The content of the last paragraph differs, particularly between older and newer reviews. 

Addressed by:
- Scraping all three paragraphs
- In next phase, will clean using header contained in first part of the paragraph "Blind Assessment", "Notes", "The Bottom Line" and "Who Should Drink It"

## Scraping Data from Individual Review Pages

Notes on scraping process:
- Scraping was a slow process, so pulled in chunks
- Older reviews were missing some information; with the "try"/"except", many of the older reviews were skipped, so adjusted scraping request and reran to increase number of unique reviews gathered. 


### Scraping First Portion of Data

In [44]:
data = [] #placeholder for storing list

In [45]:
#this function scrapes all potential data from each review page

def full_scraper(a, b):

    for url in url_list[a:b]: #indexing on url_list
        req = Request(
            url,  
            headers={'User-Agent': 'Mozilla/5.0'}
)
        page = urlopen(req).read()
        soup = bs(page, 'html.parser')
    
        try: #needed to avoid index errors related to indexing on "td" and "p"
            coffee_name = soup.find("h1", class_="review-title").text
            roaster_name = soup.find("p", class_="review-roaster").text
            roaster_location = soup.find_all("td")[1].text
            coffee_origin = soup.find_all('td')[3].text
            roast_level = soup.find_all('td')[5].text
            agtron = soup.find_all('td')[7].text
            est_price = soup.find_all('td')[9].text
            review_date = soup.find_all("td")[11].text
            overall_score = soup.find_all("span")[63].text
            h1 = soup.find_all("td")[12].text
            s1 = soup.find_all("td")[13].text
            h2 = soup.find_all("td")[14].text
            s2 = soup.find_all("td")[15].text
            h3 = soup.find_all("td")[16].text
            s3 = soup.find_all("td")[17].text.replace("\t\t\t\t\t\t\t","")
            h4 = soup.find_all("td")[18].text
            s4 = soup.find_all("td")[19].text
            h5 = soup.find_all("td")[20].text
            s5 = soup.find_all("td")[21].text
            p1 = soup.find_all("p")[16].text
            p2 = soup.find_all("p")[17].text
            p3 = soup.find_all("p")[18].text
        except IndexError:
            pass #let's the scrape continue with index error happening
            continue
    
        data.append([coffee_name, roaster_name, roaster_location, coffee_origin, roast_level, agtron, est_price, review_date, overall_score, h1, s1, h2, s2, h3, s3, h4, s4, h5, s5, p1 , p2, p3]) # adds scraped info to list
        print(f'Processed {url}!') #tracking progress
        time.sleep(random.randint(1,5)) #avoiding overloading server, trying to avoid getting blocked
    
    print(len(data))

In [46]:
full_scraper(0,1000)

Processed https://www.coffeereview.com/review/colombia-cerro-azul-enano-3/!
Processed https://www.coffeereview.com/review/peru-incahuasi/!
Processed https://www.coffeereview.com/review/colombia-apontes-guardians/!
Processed https://www.coffeereview.com/review/nicaragua-flor-de-dalia-natural/!
Processed https://www.coffeereview.com/review/ethiopia-bench-maji-geisha-g1-natural/!
Processed https://www.coffeereview.com/review/reina-de-la-asuncion-ecuador-2/!
Processed https://www.coffeereview.com/review/ethiopia-guji-uraga-more-floral-kudume-washed/!
Processed https://www.coffeereview.com/review/ethiopia-guji-uraga-more-floral-kudume-natural/!
Processed https://www.coffeereview.com/review/kenya-nyeri-hill/!
Processed https://www.coffeereview.com/review/blue-ridge-blend-2/!
Processed https://www.coffeereview.com/review/kenya-aa-top-nairobi-rose/!
Processed https://www.coffeereview.com/review/worry-free-espresso/!
Processed https://www.coffeereview.com/review/purple-tipsy-night/!
Processed h

Processed https://www.coffeereview.com/review/honeyed-floral-ethiopia-washed-guji-uraga-tome-heirloom-g1/!
Processed https://www.coffeereview.com/review/karamo-ethiopia-sidama-natural-g1/!
Processed https://www.coffeereview.com/review/caramel-rocher/!
Processed https://www.coffeereview.com/review/costa-rica-hacienda-la-minita/!
Processed https://www.coffeereview.com/review/rwanda-karambi-station-natural/!
Processed https://www.coffeereview.com/review/lone-peak-cafe-series-blend/!
Processed https://www.coffeereview.com/review/el-salvador-aida-batlle-la-florida-natural/!
Processed https://www.coffeereview.com/review/colombia-geisha-cordillera/!
Processed https://www.coffeereview.com/review/ethiopia-raro-boda/!
Processed https://www.coffeereview.com/review/kenya-nyeri-gathaithi/!
Processed https://www.coffeereview.com/review/honduras-marcala-2/!
Processed https://www.coffeereview.com/review/costa-rica-hacienda-sonora-honey/!
Processed https://www.coffeereview.com/review/ethiopia-shantawan

Processed https://www.coffeereview.com/review/brazil-mogiana-aparecid-farm/!
Processed https://www.coffeereview.com/review/costa-rica-canet-musician-series-bach-raisin-honey/!
Processed https://www.coffeereview.com/review/peru-amazonas-los-santos-geisha-natural/!
Processed https://www.coffeereview.com/review/ethiopia-odola-washed/!
Processed https://www.coffeereview.com/review/minanga-village-sulawesi/!
Processed https://www.coffeereview.com/review/kau-tropical-punch-washed/!
Processed https://www.coffeereview.com/review/kenya-peaberry-top-hill-farm-espresso/!
Processed https://www.coffeereview.com/review/birambo-village-dr-congo-2/!
Processed https://www.coffeereview.com/review/kenya-kamgogo-small-lot-reserve/!
Processed https://www.coffeereview.com/review/ethiopia-belayneh-bariso-natural/!
Processed https://www.coffeereview.com/review/kuta-kofi-papua-new-guinea/!
Processed https://www.coffeereview.com/review/rwanda-gatare-natural/!
Processed https://www.coffeereview.com/review/rock-t

Processed https://www.coffeereview.com/review/honduras-comsa-natural/!
Processed https://www.coffeereview.com/review/shakiso-natural/!
Processed https://www.coffeereview.com/review/cerro-azul-geisha-honey-process/!
Processed https://www.coffeereview.com/review/la-esmeralda-diamond-mountain-washed-coffee/!
Processed https://www.coffeereview.com/review/yirgacheffe-specialty-coffee/!
Processed https://www.coffeereview.com/review/guatemala-single-origin-espresso/!
Processed https://www.coffeereview.com/review/coe-sidama-bona-74110/!
Processed https://www.coffeereview.com/review/ethiopia-yirgacheffe-adorsi-g1/!
Processed https://www.coffeereview.com/review/tiger-blend-espresso/!
Processed https://www.coffeereview.com/review/gayo-mandheling-specialty-coffee/!
Processed https://www.coffeereview.com/review/ethiopia-yirgacheffe-g1-2/!
Processed https://www.coffeereview.com/review/karen-j-kona-red-bourbon-3/!
Processed https://www.coffeereview.com/review/el-salvador-finca-mileydi-pacamara/!
Proc

Processed https://www.coffeereview.com/review/kenya-nyeri-gatomboya-2/!
Processed https://www.coffeereview.com/review/subrosa-ethiopia-kayon-mountain-organic-natural/!
Processed https://www.coffeereview.com/review/ethiopia-bench-maji-natural-gesha/!
Processed https://www.coffeereview.com/review/panama-hacienda-la-esmeralda-gesha-3/!
Processed https://www.coffeereview.com/review/natural-kau-maragogipe/!
Processed https://www.coffeereview.com/review/costa-rica-volcan-azul-san-isidro-red-honey/!
Processed https://www.coffeereview.com/review/ethiopia-demeka-becha-2/!
Processed https://www.coffeereview.com/review/sweet-holiday-blend-3/!
Processed https://www.coffeereview.com/review/ethiopia-washed-sidama-bensa-bombe/!
Processed https://www.coffeereview.com/review/panama-abu-geisha-lot-6/!
Processed https://www.coffeereview.com/review/panama-finca-mi-finquita-geisha-afrodita-natural/!
Processed https://www.coffeereview.com/review/panama-el-burro-estate-natural-gesha/!
Processed https://www.c

Processed https://www.coffeereview.com/review/yemen-sheba-auction-lot-1-wadi-al-mahjr/!
Processed https://www.coffeereview.com/review/yemen-al-wadi/!
Processed https://www.coffeereview.com/review/yemen-port-of-mokha-al-jabal/!
Processed https://www.coffeereview.com/review/yemen-mokha-matari/!
Processed https://www.coffeereview.com/review/yemen-mocha-haimi-2/!
Processed https://www.coffeereview.com/review/yemen-sanaa/!
Processed https://www.coffeereview.com/review/bait-alal-community/!
Processed https://www.coffeereview.com/review/port-of-mokha-auction-lot-19-mohamed-ahmed-al-rumaim/!
Processed https://www.coffeereview.com/review/peru-finca-churupampa/!
Processed https://www.coffeereview.com/review/colombia-la-esperanza-potosi-anaerobic-process/!
Processed https://www.coffeereview.com/review/honduras-finca-las-moras-parainema-yellow-honey/!
Processed https://www.coffeereview.com/review/blossom-blend-espresso/!
Processed https://www.coffeereview.com/review/ethiopia-guji-shakisso-mormora-

Processed https://www.coffeereview.com/review/chocolate-lovers-espresso/!
Processed https://www.coffeereview.com/review/dharma-espresso-blend-2/!
Processed https://www.coffeereview.com/review/sudan-rume-2/!
Processed https://www.coffeereview.com/review/tanzania-gesha-peaberry/!
Processed https://www.coffeereview.com/review/panama-geisha-4/!
Processed https://www.coffeereview.com/review/ethiopia-natural-guji-d-minor-21-01/!
Processed https://www.coffeereview.com/review/ethiopia-hambela-yellow-honey-james-selection/!
Processed https://www.coffeereview.com/review/costa-rica-black-honey-2/!
Processed https://www.coffeereview.com/review/ethiopia-guji-hambela-kudume-yellow-honey/!
Processed https://www.coffeereview.com/review/esmeralda-estate-panama-geisha-4/!
Processed https://www.coffeereview.com/review/ethiopia-sidamo-bensa-mirado-natural/!
Processed https://www.coffeereview.com/review/costa-rica-la-florida-red-honey-espresso/!
Processed https://www.coffeereview.com/review/ethiopia-yirgac

Processed https://www.coffeereview.com/review/rwanda-nyamagabe-kigeme-lot-4/!
Processed https://www.coffeereview.com/review/la-estrella-carbonic-geisha-reserve/!
Processed https://www.coffeereview.com/review/ethiopia-guji-wush-wush-anaerobic-natural/!
Processed https://www.coffeereview.com/review/ethiopia-guji-wush-wush/!
Processed https://www.coffeereview.com/review/tres-dragones-colombia/!
Processed https://www.coffeereview.com/review/panama-elida-estate-catuai-natural-asd/!
Processed https://www.coffeereview.com/review/ethiopia-guji-natural-anaerobic-wush-wush-g1/!
Processed https://www.coffeereview.com/review/peru-yanesha-geisha-double-anaerobic/!
Processed https://www.coffeereview.com/review/elida-natural-asd-panama/!
Processed https://www.coffeereview.com/review/pink-bourbon-natural-anaerobic-colombia/!
Processed https://www.coffeereview.com/review/ethiopia-oromia-anaerobic-natural/!
Processed https://www.coffeereview.com/review/villa-loyola-carbonic-maceration/!
Processed https:

Processed https://www.coffeereview.com/review/espresso-15/!
Processed https://www.coffeereview.com/review/estate-finca-la-lina/!
Processed https://www.coffeereview.com/review/5s-select-coffee-bean/!
Processed https://www.coffeereview.com/review/ethiopia-neja-fadil/!
Processed https://www.coffeereview.com/review/congo-muungano-3/!
Processed https://www.coffeereview.com/review/ethiopia-guji-hambela-g1-natural/!
Processed https://www.coffeereview.com/review/peru-buena-vista-valerio-almanza-ccahua/!
Processed https://www.coffeereview.com/review/private-reserve-ethiopia-barrel-aged/!
Processed https://www.coffeereview.com/review/private-reserve-costa-rica-naranjo/!
Processed https://www.coffeereview.com/review/private-reserve-guatemala-medina-estate/!
Processed https://www.coffeereview.com/review/panama-finca-santa-teresa-lot-9/!
Processed https://www.coffeereview.com/review/honduro-spro/!
Processed https://www.coffeereview.com/review/guatemala-coban-chicoj-cooperative-red-honey-fair-trade/

Processed https://www.coffeereview.com/review/guatemala-acatenango/!
Processed https://www.coffeereview.com/review/kenya-gachatha-ab/!
Processed https://www.coffeereview.com/review/coral/!
Processed https://www.coffeereview.com/review/zafiro/!
Processed https://www.coffeereview.com/review/columbia-la-esperanza-winey-natural/!
Processed https://www.coffeereview.com/review/costa-rica-cumbres-del-poas-anaerobic/!
Processed https://www.coffeereview.com/review/super-instant-coffee/!
Processed https://www.coffeereview.com/review/costa-rica-las-lajas-perla-negra/!
Processed https://www.coffeereview.com/review/papua-new-guinea-kenta/!
Processed https://www.coffeereview.com/review/ethiopia-guji-goro-muda-washed-g1/!
Processed https://www.coffeereview.com/review/ethiopian-guji-ana-sora-alcoholic-natural-g1/!
Processed https://www.coffeereview.com/review/rum-barrel-aged-kauai-coffee/!
Processed https://www.coffeereview.com/review/panama-hacienda-la-esmeralda-gesha-2/!
Processed https://www.coffee

In [47]:
full_scraper(1000,2000)

Processed https://www.coffeereview.com/review/thailand-omkoi-sopas-estate/!
Processed https://www.coffeereview.com/review/espresso-blend-20/!
Processed https://www.coffeereview.com/review/burundi-buhorwa-3/!
Processed https://www.coffeereview.com/review/monte-alban-2/!
Processed https://www.coffeereview.com/review/cual-bicicleta/!
Processed https://www.coffeereview.com/review/pena-blanca-black-honey/!
Processed https://www.coffeereview.com/review/la-joya/!
Processed https://www.coffeereview.com/review/magic-cat-kenya-civet-afca-toh-no-2-gatuya-factory-new-murarandia-aa/!
Processed https://www.coffeereview.com/review/magic-cat-civet-ethiopia-sidamo-shantwene-litchi-rose-washed/!
Processed https://www.coffeereview.com/review/ethiopia-grade-1-chelchele/!
Processed https://www.coffeereview.com/review/ethiopia-natural-yirgacheffe-gr-1-cleopatra/!
Processed https://www.coffeereview.com/review/taiwan-alishan-zhou-zhu-yuan-farm-estate-reserve-washed/!
Processed https://www.coffeereview.com/rev

Processed https://www.coffeereview.com/review/kenya-ares/!
Processed https://www.coffeereview.com/review/empress-kenya/!
Processed https://www.coffeereview.com/review/kenya-kiambu-ab-top-lot/!
Processed https://www.coffeereview.com/review/kenya-kiambu-maua-lot-ab/!
Processed https://www.coffeereview.com/review/kenya-thunguri-aa/!
Processed https://www.coffeereview.com/review/kenya-kirinyaga-kabare-karani-aa/!
Processed https://www.coffeereview.com/review/kenya-kirinyaga-kii/!
Processed https://www.coffeereview.com/review/kenya-rungeto/!
Processed https://www.coffeereview.com/review/kenya-aa-top-gatomboya/!
Processed https://www.coffeereview.com/review/honduras-2/!
Processed https://www.coffeereview.com/review/heart-soul-espresso-blend/!
Processed https://www.coffeereview.com/review/ethiopia-banko-gotiti-3/!
Processed https://www.coffeereview.com/review/ethiopia-yabitu-koba/!
Processed https://www.coffeereview.com/review/ethiopia-banko-gotiti-2/!
Processed https://www.coffeereview.com/r

Processed https://www.coffeereview.com/review/nicaragua-la-benedicion-sl28-natural/!
Processed https://www.coffeereview.com/review/colombia-tolima-geisha/!
Processed https://www.coffeereview.com/review/classic-blend-fto-rfa/!
Processed https://www.coffeereview.com/review/galapagos-la-tortuga/!
Processed https://www.coffeereview.com/review/south-africa-elephant-coast-estate-coffea-racemosa/!
Processed https://www.coffeereview.com/review/titan-blend-fto/!
Processed https://www.coffeereview.com/review/ethiopia-guji-hambela-buku-abel-natural-g1/!
Processed https://www.coffeereview.com/review/ethiopia-haro-guta-natural/!
Processed https://www.coffeereview.com/review/ethiopia-yirgacheffe-hafursa-g1/!
Processed https://www.coffeereview.com/review/kenya-aa-kirinyaga-kii/!
Processed https://www.coffeereview.com/review/colombia-antioquia-la-papaya-bourbon-chiroso/!
Processed https://www.coffeereview.com/review/ethiopia-gatame-muka-natural/!
Processed https://www.coffeereview.com/review/geisha-ve

Processed https://www.coffeereview.com/review/gesha-village-single-variety-gesha-special-selection/!
Processed https://www.coffeereview.com/review/dr-congo-muungano/!
Processed https://www.coffeereview.com/review/colombian-5/!
Processed https://www.coffeereview.com/review/magnolia-blend-2/!
Processed https://www.coffeereview.com/review/rwanda-gatare-namasheke/!
Processed https://www.coffeereview.com/review/indestec-el-paraiso/!
Processed https://www.coffeereview.com/review/monteblanco-cold-fermentation-geisha/!
Processed https://www.coffeereview.com/review/funky-chicken/!
Processed https://www.coffeereview.com/review/dirty-south/!
Processed https://www.coffeereview.com/review/ten-anniversary-blend/!
Processed https://www.coffeereview.com/review/420-french-roast/!
Processed https://www.coffeereview.com/review/19793/!
Processed https://www.coffeereview.com/review/floyd-farmhouse-blend/!
Processed https://www.coffeereview.com/review/yemen-al-obbarat/!
Processed https://www.coffeereview.co

Processed https://www.coffeereview.com/review/ecuador-eugenioides-natural-finca-perla-negra/!
Processed https://www.coffeereview.com/review/panama-esmeralda-geisha-natural/!
Processed https://www.coffeereview.com/review/ethiopia-suke-quto-honey-2/!
Processed https://www.coffeereview.com/review/single-origin-espresso/!
Processed https://www.coffeereview.com/review/kenya-fsc-ab/!
Processed https://www.coffeereview.com/review/colombia-pink-bourbon-2/!
Processed https://www.coffeereview.com/review/guatemala-acatenango-geisha/!
Processed https://www.coffeereview.com/review/kenya-kiambu-summer-love/!
Processed https://www.coffeereview.com/review/el-salvador-pacamara-honey/!
Processed https://www.coffeereview.com/review/latte-espresso/!
Processed https://www.coffeereview.com/review/blend-espresso/!
Processed https://www.coffeereview.com/review/ceri-ceri-meriah-coffee/!
Processed https://www.coffeereview.com/review/alamendah-patuha-coffee/!
Processed https://www.coffeereview.com/review/america

Processed https://www.coffeereview.com/review/best-of-panama-gnep-01-elida-geisha-green-tip-natural/!
Processed https://www.coffeereview.com/review/geisha-xo/!
Processed https://www.coffeereview.com/review/colombia-finca-la-maria-geisha-natural/!
Processed https://www.coffeereview.com/review/panama-finca-kalithea-natural-geisha/!
Processed https://www.coffeereview.com/review/panama-hacienda-la-esmeralda-super-mario-6-geisha/!
Processed https://www.coffeereview.com/review/panama-geisha-alto-jaramillo/!
Processed https://www.coffeereview.com/review/colombia-laderas-del-tapias-estate-natural-geisha/!
Processed https://www.coffeereview.com/review/gesha-village-natural-lot-19/!
Processed https://www.coffeereview.com/review/gesha-village-1931-lot-27/!
Processed https://www.coffeereview.com/review/ethiopia-guji-girma-natural/!
Processed https://www.coffeereview.com/review/ethiopia-limu-lema-edeto/!
Processed https://www.coffeereview.com/review/ethiopia-sidamo-gora-kone/!
Processed https://www

Processed https://www.coffeereview.com/review/maui-kupaa-orange-bourbon/!
Processed https://www.coffeereview.com/review/honey-typica/!
Processed https://www.coffeereview.com/review/kau-typica-natural/!
Processed https://www.coffeereview.com/review/estate-grown-mamo/!
Processed https://www.coffeereview.com/review/australian-blend-espresso/!
Processed https://www.coffeereview.com/review/colombian-supremo-7/!
Processed https://www.coffeereview.com/review/colombian-supremo-6/!
Processed https://www.coffeereview.com/review/rwandan-2/!
Processed https://www.coffeereview.com/review/colombia-las-joyas-fidencio-martinez-lot/!
Processed https://www.coffeereview.com/review/colombia-tolima-monteverde-estate-natural-wush-wush/!
Processed https://www.coffeereview.com/review/ethiopia-oromia-guji-washed/!
Processed https://www.coffeereview.com/review/yemen-matari/!
Processed https://www.coffeereview.com/review/kenya-muranga-githiga-aa/!
Processed https://www.coffeereview.com/review/borneo-sabah-arabic

Processed https://www.coffeereview.com/review/ethiopia-gugi/!
Processed https://www.coffeereview.com/review/kenya-murunga-new-gatanga-fcs-mukurwe-factory-ab/!
Processed https://www.coffeereview.com/review/brazil-recreio-selectivo/!
Processed https://www.coffeereview.com/review/ethiopia-worka-2/!
Processed https://www.coffeereview.com/review/guatemala-hunapu/!
Processed https://www.coffeereview.com/review/ethiopia-banko-dhadhato-washed/!
Processed https://www.coffeereview.com/review/kenya-rugeto-peaberry/!
Processed https://www.coffeereview.com/review/gayo-aceh-sumatra/!
Processed https://www.coffeereview.com/review/kenya-karuthi-aa/!
Processed https://www.coffeereview.com/review/kenya-nyeri-nudurutu-factory-ab/!
Processed https://www.coffeereview.com/review/ethiopia-kochere-washed-saona-lot/!
Processed https://www.coffeereview.com/review/ethiopia-guji-natural-dasaya-se-1/!
Processed https://www.coffeereview.com/review/auromar-geisha-natural-2/!
Processed https://www.coffeereview.com/re

Processed https://www.coffeereview.com/review/alemu-bukato-espresso/!
Processed https://www.coffeereview.com/review/alemu-bukato-ethiopia/!
Processed https://www.coffeereview.com/review/asopep-colombia/!
Processed https://www.coffeereview.com/review/tarime-tanzania-2/!
Processed https://www.coffeereview.com/review/guachepelin-costa-rica/!
Processed https://www.coffeereview.com/review/berlina-geisha/!
Processed https://www.coffeereview.com/review/gatugi-triple/!
Processed https://www.coffeereview.com/review/sulawesi-toarco/!
Processed https://www.coffeereview.com/review/ethiopia-yirgacheffe-natural-espresso/!
Processed https://www.coffeereview.com/review/ethiopia-yirgacheffe-natural/!
Processed https://www.coffeereview.com/review/guatemala-finca-el-general-lot-alt8-espresso/!
Processed https://www.coffeereview.com/review/green-dolphin-blend-espresso/!
Processed https://www.coffeereview.com/review/h-coffee-espresso-blend-01/!
Processed https://www.coffeereview.com/review/brazil-conquista

In [49]:
full_scraper(2000,3000)

Processed https://www.coffeereview.com/review/mahonda-burundi/!
Processed https://www.coffeereview.com/review/signature-hazelnut-dark-chocolate/!
Processed https://www.coffeereview.com/review/signature-floral-orange/!
Processed https://www.coffeereview.com/review/el-salvador-6/!
Processed https://www.coffeereview.com/review/kiniyota-espresso/!
Processed https://www.coffeereview.com/review/guardian-spirit-espresso-blend/!
Processed https://www.coffeereview.com/review/kenya-nyeri-rumukia-peaberry/!
Processed https://www.coffeereview.com/review/kenya-nyeri-karuthi-aa/!
Processed https://www.coffeereview.com/review/el-obraje-geisha-2/!
Processed https://www.coffeereview.com/review/burundi-2/!
Processed https://www.coffeereview.com/review/mirador-colombia/!
Processed https://www.coffeereview.com/review/kenya-kirinyaga-mukangu-ab-2/!
Processed https://www.coffeereview.com/review/brazil-carmo-de-minas-natural/!
Processed https://www.coffeereview.com/review/brazil-pulped-natural/!
Processed ht

Processed https://www.coffeereview.com/review/ethiopia-reko-estate-yirgacheffe/!
Processed https://www.coffeereview.com/review/costa-rica-sonora-estate-venecia-natural/!
Processed https://www.coffeereview.com/review/mini-washed-kaffa-forest/!
Processed https://www.coffeereview.com/review/jade-espresso/!
Processed https://www.coffeereview.com/review/estate-medium-roast/!
Processed https://www.coffeereview.com/review/ethiopia-holy-grail-natural-yirgacheffe/!
Processed https://www.coffeereview.com/review/ethiopia-agakiso/!
Processed https://www.coffeereview.com/review/kenya-karinga-3/!
Processed https://www.coffeereview.com/review/espresso-classico-7/!
Processed https://www.coffeereview.com/review/thailand-doi-pangkhon-espresso-xxx/!
Processed https://www.coffeereview.com/review/ulos-batak-sumatra-3/!
Processed https://www.coffeereview.com/review/gaitania-colombia/!
Processed https://www.coffeereview.com/review/blue-label-la-esmeralda-porton-pascua-gesha/!
Processed https://www.coffeerevi

Processed https://www.coffeereview.com/review/jinotega-geisha/!
Processed https://www.coffeereview.com/review/bernachon-b888-espresso/!
Processed https://www.coffeereview.com/review/ho-soo-tsais-g-e-duetto/!
Processed https://www.coffeereview.com/review/gedeb-lot-83-ethiopia-natural/!
Processed https://www.coffeereview.com/review/halo-beriti-ethiopia/!
Processed https://www.coffeereview.com/review/mutitu-kenya/!
Processed https://www.coffeereview.com/review/guayab-guatemala/!
Processed https://www.coffeereview.com/review/el-aquila-pacamara-el-salvador/!
Processed https://www.coffeereview.com/review/ethiopia-konga-sede/!
Processed https://www.coffeereview.com/review/colombia-la-guamera/!
Processed https://www.coffeereview.com/review/static-amaro-gayo-natural-ethiopia/!
Processed https://www.coffeereview.com/review/static-cali-classico-espresso/!
Processed https://www.coffeereview.com/review/static-dark-star-mexico-espresso/!
Processed https://www.coffeereview.com/review/static-hacienda-

Processed https://www.coffeereview.com/review/kenya-marumi-aa/!
Processed https://www.coffeereview.com/review/competition-espresso/!
Processed https://www.coffeereview.com/review/espresso-blend-01-full-city-roast/!
Processed https://www.coffeereview.com/review/burundi-kayanza-kabuye-mpanga-natural/!
Processed https://www.coffeereview.com/review/100-arabica-coffee-puerto-rico/!
Processed https://www.coffeereview.com/review/gesha-village-estate-bench-maji-gesha-g1/!
Processed https://www.coffeereview.com/review/pink-bourbon-colombia-cauca/!
Processed https://www.coffeereview.com/review/barrel-aged-coffee/!
Processed https://www.coffeereview.com/review/wine-barrel-aged-ethiopia/!
Processed https://www.coffeereview.com/review/bourbon-barrel-aged-kenya-baragu/!
Processed https://www.coffeereview.com/review/pinot-noir-aged-el-salvador/!
Processed https://www.coffeereview.com/review/puna-theobroma-coffee/!
Processed https://www.coffeereview.com/review/aged-sumatra-ulos-batak-peaberry/!
Proces

Processed https://www.coffeereview.com/review/kahiko-holiday-coffee/!
Processed https://www.coffeereview.com/review/gigisa-ethiopia/!
Processed https://www.coffeereview.com/review/poinsettia-blend/!
Processed https://www.coffeereview.com/review/the-jungle/!
Processed https://www.coffeereview.com/review/natural-brazil-hat-magic-phoenix-special/!
Processed https://www.coffeereview.com/review/ethiopia-yirgacheffe-natural-celinga-foge-grade-1/!
Processed https://www.coffeereview.com/review/costa-rica-las-lajas-alma-negra/!
Processed https://www.coffeereview.com/review/colombian-4/!
Processed https://www.coffeereview.com/review/costa-rica-medium-roast/!
Processed https://www.coffeereview.com/review/christmas-blend-5/!
Processed https://www.coffeereview.com/review/ethiopia-yirgacheffe-23/!
Processed https://www.coffeereview.com/review/panama-santa-teresa-natural-2/!
Processed https://www.coffeereview.com/review/light-roast-2/!
Processed https://www.coffeereview.com/review/ethiopia-washed-yir

Processed https://www.coffeereview.com/review/worka-ethiopia/!
Processed https://www.coffeereview.com/review/waykan-guatemala-2/!
Processed https://www.coffeereview.com/review/worka-ethiopia-espresso/!
Processed https://www.coffeereview.com/review/hambela-natural/!
Processed https://www.coffeereview.com/review/honduras-natural/!
Processed https://www.coffeereview.com/review/kenya-kiamabara-2/!
Processed https://www.coffeereview.com/review/gisuma-rwanda/!
Processed https://www.coffeereview.com/review/ethiopia-zonegediyo-gedeb/!
Processed https://www.coffeereview.com/review/costa-rica-tarrazu-3/!
Processed https://www.coffeereview.com/review/indonesia-sumatra-tano-batak/!
Processed https://www.coffeereview.com/review/mexico-canada-fria/!
Processed https://www.coffeereview.com/review/selian-tanzania-2/!
Processed https://www.coffeereview.com/review/yemen-rayyan-mill/!
Processed https://www.coffeereview.com/review/el-salvador-finca-loma-la-gloria-2/!
Processed https://www.coffeereview.com/

Processed https://www.coffeereview.com/review/ethiopia-sidama-deri-kochoha-wp-2/!
Processed https://www.coffeereview.com/review/validation-espresso-blend/!
Processed https://www.coffeereview.com/review/brazil-fazenda-passeio-natural/!
Processed https://www.coffeereview.com/review/brazil-estancia-telese/!
Processed https://www.coffeereview.com/review/fazenda-sertao-brazil/!
Processed https://www.coffeereview.com/review/fazenda-aurea-south-minas-brazil/!
Processed https://www.coffeereview.com/review/brazil-fazenda-do-serrado/!
Processed https://www.coffeereview.com/review/carmo-de-minas-canaan-estate-natural-pedreira-microlot/!
Processed https://www.coffeereview.com/review/carmo-de-minas-oliveira-fernando-manoel-de-oliveira-brazil-natural/!
Processed https://www.coffeereview.com/review/boa-vista-brazil/!
Processed https://www.coffeereview.com/review/brazil-fazenda-beneficio-presente-do-sol/!
Processed https://www.coffeereview.com/review/milepost-zimbabwe-aa-smaldeel-estate/!
Processed ht

Processed https://www.coffeereview.com/review/rwanda-butare/!
Processed https://www.coffeereview.com/review/tanzania-kikafu-estate-direct-trade-light-roast/!
Processed https://www.coffeereview.com/review/colombia-luminosa-light-roast/!
Processed https://www.coffeereview.com/review/big-trouble/!
Processed https://www.coffeereview.com/review/100-colombian-ground-coffee/!
Processed https://www.coffeereview.com/review/100-colombian-medium-roast/!
Processed https://www.coffeereview.com/review/kenya-kii/!
Processed https://www.coffeereview.com/review/congo-sopacdi/!
Processed https://www.coffeereview.com/review/ethiopia-yirgacheffe-dry-process/!
Processed https://www.coffeereview.com/review/mis-blend/!
Processed https://www.coffeereview.com/review/danceing-blend/!
Processed https://www.coffeereview.com/review/kenya-aa-nyeri-tambaya/!
Processed https://www.coffeereview.com/review/costa-rica-las-lajas-red-honey/!
Processed https://www.coffeereview.com/review/rwanda-kabirizi-organic/!
Processed

Processed https://www.coffeereview.com/review/fto-guatemala-huehuetenango/!
Processed https://www.coffeereview.com/review/honduras-finca-gaby/!
Processed https://www.coffeereview.com/review/423-blend/!
Processed https://www.coffeereview.com/review/panama-altieri-natural-geisha/!
Processed https://www.coffeereview.com/review/ethiopia-adado-sulula/!
Processed https://www.coffeereview.com/review/greenwell-estate-hawaii-kona-natural-espresso/!
Processed https://www.coffeereview.com/review/greenwell-estate-hawaii-kona-peaberry-natural/!
Processed https://www.coffeereview.com/review/organic-espresso-2/!
Processed https://www.coffeereview.com/review/organic-holiday-blend/!
Processed https://www.coffeereview.com/review/bean-brain-reserve-perla-negra-costa-rica/!
Processed https://www.coffeereview.com/review/kemgin-ninety-plus/!
Processed https://www.coffeereview.com/review/ethiopia-nigusse-lemma/!
Processed https://www.coffeereview.com/review/colombia-maximino-gutierrez/!
Processed https://www

2993


In [50]:
full_scraper(3000,4000)

Processed https://www.coffeereview.com/review/ethiopian-yirgacheffe-kela-kochore/!
Processed https://www.coffeereview.com/review/ethiopia-yirgacheffe-dabub-matebya/!
Processed https://www.coffeereview.com/review/yemen-haraaz-red-a/!
Processed https://www.coffeereview.com/review/guatemala-waykan/!
Processed https://www.coffeereview.com/review/el-salvador-finca-loma-la-gloria/!
Processed https://www.coffeereview.com/review/guatemala-finca-santa-isabel-2/!
Processed https://www.coffeereview.com/review/ethiopia-sidama-ardi-4/!
Processed https://www.coffeereview.com/review/kenya-karagoto-2/!
Processed https://www.coffeereview.com/review/rwanda-nyanza-kirezi/!
Processed https://www.coffeereview.com/review/colombia-caldas-laderas-del-tapias/!
Processed https://www.coffeereview.com/review/brazil-sitio-baixadao/!
Processed https://www.coffeereview.com/review/organic-colombia-aaa-microlot/!
Processed https://www.coffeereview.com/review/burundi-kayanza-ninga/!
Processed https://www.coffeereview.c

Processed https://www.coffeereview.com/review/rwanda-bufcafe-bourbon-lot/!
Processed https://www.coffeereview.com/review/kenya-gakui-peaberry/!
Processed https://www.coffeereview.com/review/guatemala-medium-roast/!
Processed https://www.coffeereview.com/review/rwanda-kinyaga-co-op/!
Processed https://www.coffeereview.com/review/ethiopia-washed-yirgacheffe-worka-coop-alemu-2/!
Processed https://www.coffeereview.com/review/ethiopia-washed-yirgacheffe-worka-coop-alemu/!
Processed https://www.coffeereview.com/review/rwanda-kanzu-lot-13/!
Processed https://www.coffeereview.com/review/kenya-kianjuri-ab/!
Processed https://www.coffeereview.com/review/kenya-nyeri-tambaya-peaberry/!
Processed https://www.coffeereview.com/review/ethiopian-yirgacheffe-idido-natural/!
Processed https://www.coffeereview.com/review/espresso-crema-dolce/!
Processed https://www.coffeereview.com/review/ethiopia-yirgacheffe-konga-natural-single-serve-capsule/!
Processed https://www.coffeereview.com/review/fifth-avenue-m

Processed https://www.coffeereview.com/review/ethiopia-homacho-waeno-2/!
Processed https://www.coffeereview.com/review/colombia-granja-esperanza-gesha-aaa/!
Processed https://www.coffeereview.com/review/ethiopia-homacho-waeno/!
Processed https://www.coffeereview.com/review/guatemala-finca-el-valle-antigua/!
Processed https://www.coffeereview.com/review/ethiopia-wotona-bultuma/!
Processed https://www.coffeereview.com/review/mig-washing-station-gikongoro-rwanda/!
Processed https://www.coffeereview.com/review/rwanda-buremura/!
Processed https://www.coffeereview.com/review/nicaragua-las-hermanas-fto/!
Processed https://www.coffeereview.com/review/mexico-fto/!
Processed https://www.coffeereview.com/review/los-guardianes-peru-fto/!
Processed https://www.coffeereview.com/review/honey-margarita-colombia/!
Processed https://www.coffeereview.com/review/kenya-mathew-mugo/!
Processed https://www.coffeereview.com/review/organic-ethiopian-banko-natural/!
Processed https://www.coffeereview.com/review

Processed https://www.coffeereview.com/review/guatemala-asdecafe/!
Processed https://www.coffeereview.com/review/ethiopia-kochere/!
Processed https://www.coffeereview.com/review/rwanda-hinga-kawa/!
Processed https://www.coffeereview.com/review/mexico-cesmach-fto/!
Processed https://www.coffeereview.com/review/burundi-mpanga/!
Processed https://www.coffeereview.com/review/mexican-vera-cruz-finca-kassandra/!
Processed https://www.coffeereview.com/review/las-margaritas-colombia-gesha/!
Processed https://www.coffeereview.com/review/el-diamante-huehuetenango-guatemala/!
Processed https://www.coffeereview.com/review/ethiopia-yirgacheffe-kore-kochore/!
Processed https://www.coffeereview.com/review/sombra-del-poro-costa-rica/!
Processed https://www.coffeereview.com/review/so-espresso-brazil-fazenda-rainha-natural/!
Processed https://www.coffeereview.com/review/ethiopia-natural-yirgacheffe-reko/!
Processed https://www.coffeereview.com/review/summit-premium-espresso-blend/!
Processed https://www

Processed https://www.coffeereview.com/review/testa-rossa-classic-espresso-blend/!
Processed https://www.coffeereview.com/review/aricha-ethiopia-yirgacheffe-natural-process/!
Processed https://www.coffeereview.com/review/kenya-makwa-ab-2/!
Processed https://www.coffeereview.com/review/kabirizi-rwanda/!
Processed https://www.coffeereview.com/review/wahana-sumatra/!
Processed https://www.coffeereview.com/review/tolima-colombia/!
Processed https://www.coffeereview.com/review/palanda-ecuador-organic/!
Processed https://www.coffeereview.com/review/kenya-kabare-ab/!
Processed https://www.coffeereview.com/review/sumatra-wahana-natural-process/!
Processed https://www.coffeereview.com/review/peaberry-reserve/!
Processed https://www.coffeereview.com/review/honduras-santa-barbara-beneficio-san-vicente-caracolillo/!
Processed https://www.coffeereview.com/review/ferghana-horse-blend/!
Processed https://www.coffeereview.com/review/kenya-makwa-ab/!
Processed https://www.coffeereview.com/review/monk-s

Processed https://www.coffeereview.com/review/juliette-red-ninety-plus-gesha-estate-panama/!
Processed https://www.coffeereview.com/review/100-kona-high-mountain-io-extra-fancy/!
Processed https://www.coffeereview.com/review/honduras-el-cielito-finca-las-brisas/!
Processed https://www.coffeereview.com/review/thanksgiving-blend-2013/!
Processed https://www.coffeereview.com/review/rwanda-abakundakawa/!
Processed https://www.coffeereview.com/review/sumatra-blue-batak-peaberry/!
Processed https://www.coffeereview.com/review/costa-rica-sonora-estate-honey-valencia/!
Processed https://www.coffeereview.com/review/panama-los-lajones-estate-lot-14/!
Processed https://www.coffeereview.com/review/ethiopia-tswana/!
Processed https://www.coffeereview.com/review/ethiopia-worka/!
Processed https://www.coffeereview.com/review/guatemala-finca-san-jorge-maracaturra/!
Processed https://www.coffeereview.com/review/wahana-estate-sumatra-longberry-2013-microlot/!
Processed https://www.coffeereview.com/revie

Processed https://www.coffeereview.com/review/verita/!
Processed https://www.coffeereview.com/review/espresso-intenso-rivo-pack/!
Processed https://www.coffeereview.com/review/verismo-espresso-roast-pod/!
Processed https://www.coffeereview.com/review/vigore/!
Processed https://www.coffeereview.com/review/costa-rica-finca-el-quizarra-white-honey/!
Processed https://www.coffeereview.com/review/nantucket-blend-k-cup/!
Processed https://www.coffeereview.com/review/donut-shop-coffee-vue-pack/!
Processed https://www.coffeereview.com/review/s-o-espresso-rwanda-rulindo/!
Processed https://www.coffeereview.com/review/high-tide-organic-espresso/!
Processed https://www.coffeereview.com/review/s-o-espresso-organic-sumatra-aceh/!
Processed https://www.coffeereview.com/review/ethiopia-ardi/!
Processed https://www.coffeereview.com/review/organic-ethiopia-sidamo/!
Processed https://www.coffeereview.com/review/kangocho-kenya/!
Processed https://www.coffeereview.com/review/costa-rica-rio-jorco-el-llano-

Processed https://www.coffeereview.com/review/bolivia-buenavista-colonia-villa-rosario/!
Processed https://www.coffeereview.com/review/decaf-organic-papua-new-guinea/!
Processed https://www.coffeereview.com/review/chickadee-medium-roast/!
Processed https://www.coffeereview.com/review/peru-organic-puno/!
Processed https://www.coffeereview.com/review/ethiopia-ninety-plus-kemgin/!
Processed https://www.coffeereview.com/review/kenya-riakiberu-peaberry/!
Processed https://www.coffeereview.com/review/rwanda-nyamasheke/!
Processed https://www.coffeereview.com/review/el-salvador-santa-marta-honey/!
Processed https://www.coffeereview.com/review/bolivia-carmelo-yujra-organic/!
Processed https://www.coffeereview.com/review/papua-new-guinea-aa-akora/!
Processed https://www.coffeereview.com/review/hawaii-kona-isla-prime-select/!
Processed https://www.coffeereview.com/review/premium-roasted-thai-coffee-beans/!
Processed https://www.coffeereview.com/review/bolivia-organic-buenavista-1738/!
Processed 

Processed https://www.coffeereview.com/review/ethiopia-amaro-gayo-2/!
Processed https://www.coffeereview.com/review/ethiopia-dry-process-saris-abaya/!
Processed https://www.coffeereview.com/review/ethiopia-amaro-gayo/!
Processed https://www.coffeereview.com/review/idido-cooperative-yirgacheffe-ethiopia/!
Processed https://www.coffeereview.com/review/espresso-blend-7/!
Processed https://www.coffeereview.com/review/espresso-6/!
Processed https://www.coffeereview.com/review/summit-winter-blend/!
Processed https://www.coffeereview.com/review/holiday-blend-13/!
Processed https://www.coffeereview.com/review/holiday-blend-12/!
Processed https://www.coffeereview.com/review/fa-la-la-la-la/!
Processed https://www.coffeereview.com/review/colombia-san-agustin-arnulfo-leguizamo/!
Processed https://www.coffeereview.com/review/original-christmas-so-yirgacheffe/!
Processed https://www.coffeereview.com/review/ethiopia-nekisse-2/!
Processed https://www.coffeereview.com/review/rwanda-abakundakawa-2/!
Pro

In [51]:
full_scraper(4000,5000)

Processed https://www.coffeereview.com/review/costa-rican-miguel-rojas-miso/!
Processed https://www.coffeereview.com/review/signature-mocha-java/!
Processed https://www.coffeereview.com/review/barista-prima-coffeehouse-colombia-k-cup-2/!
Processed https://www.coffeereview.com/review/gichugu-kenya-espresso/!
Processed https://www.coffeereview.com/review/sidamo-guji/!
Processed https://www.coffeereview.com/review/ethiopia-yirgacheffe-ecx-top-lot-2/!
Processed https://www.coffeereview.com/review/mihuti-kenya-aa/!
Processed https://www.coffeereview.com/review/ethiopia-nanno-challa-cooperative/!
Processed https://www.coffeereview.com/review/sumatra-lake-tawar-6/!
Processed https://www.coffeereview.com/review/panama-kotowa-duncan-2012-microlot/!
Processed https://www.coffeereview.com/review/kenya-thiriku-top-auction-lot-2/!
Processed https://www.coffeereview.com/review/blue-special-blend/!
Processed https://www.coffeereview.com/review/hama-blend/!
Processed https://www.coffeereview.com/revie

Processed https://www.coffeereview.com/review/ethiopian-sidamo-shanta-golba-natural-processed/!
Processed https://www.coffeereview.com/review/brasil-fazenda-sertao/!
Processed https://www.coffeereview.com/review/brazil-zinho-peaberry-natural/!
Processed https://www.coffeereview.com/review/organic-brazil-diamantina/!
Processed https://www.coffeereview.com/review/brazilian-medium-roast/!
Processed https://www.coffeereview.com/review/brazil-fazenda-floresta/!
Processed https://www.coffeereview.com/review/brazil-monte-alegre/!
Processed https://www.coffeereview.com/review/brazil-sitio-do-tanque/!
Processed https://www.coffeereview.com/review/brazil-fazenda-chapadao-de-ferro-peaberry/!
Processed https://www.coffeereview.com/review/sol-do-paraguassu-farm-natural-processed-brazil/!
Processed https://www.coffeereview.com/review/el-vergel-guatemala/!
Processed https://www.coffeereview.com/review/colombia-finca-villa-loyola/!
Processed https://www.coffeereview.com/review/colombian-la-esperanza/!

Processed https://www.coffeereview.com/review/el-salvado-finca-ayatepeque-natural-processed-pacamara/!
Processed https://www.coffeereview.com/review/nicaragua-bourbon-city-roast/!
Processed https://www.coffeereview.com/review/guatemala-hanupu/!
Processed https://www.coffeereview.com/review/panama-geisha-aroma-roast/!
Processed https://www.coffeereview.com/review/guatemala-pandora-pacamara-medium-light-roast/!
Processed https://www.coffeereview.com/review/starlight-blend/!
Processed https://www.coffeereview.com/review/garden-espresso-blend/!
Processed https://www.coffeereview.com/review/medium-roast-yellow-bourbon-pulped-natural/!
Processed https://www.coffeereview.com/review/colombia-tolimas/!
Processed https://www.coffeereview.com/review/kenya-nyeri-ab-gichatha-ini-signature-series/!
Processed https://www.coffeereview.com/review/kenya-aa-nyeri-ndiaini-espresso/!
Processed https://www.coffeereview.com/review/ethiopia-harrar-3/!
Processed https://www.coffeereview.com/review/ethiopia-yir

Processed https://www.coffeereview.com/review/kenya-gichatha-ini-2/!
Processed https://www.coffeereview.com/review/kenya-gichatha-ini/!
Processed https://www.coffeereview.com/review/ethiopia-yirgacheffe-koke-coop-fair-trade-organic/!
Processed https://www.coffeereview.com/review/nicaragua-cinco-de-junio-fair-trade-organic/!
Processed https://www.coffeereview.com/review/panama-gesha-2011/!
Processed https://www.coffeereview.com/review/rwanda-coopac-3/!
Processed https://www.coffeereview.com/review/guatemala-xeucalvitz-2/!
Processed https://www.coffeereview.com/review/kenya-karie-micro-lot/!
Processed https://www.coffeereview.com/review/guatemala-guayab-3/!
Processed https://www.coffeereview.com/review/guatemala-xeucalvitz-espresso-2/!
Processed https://www.coffeereview.com/review/rwanda-musasa/!
Processed https://www.coffeereview.com/review/mexico-chiapas-fair-trade/!
Processed https://www.coffeereview.com/review/colombian-supremo-5/!
Processed https://www.coffeereview.com/review/kenya-

Processed https://www.coffeereview.com/review/rwanda-musasa-red-bourbon/!
Processed https://www.coffeereview.com/review/espresso-luna/!
Processed https://www.coffeereview.com/review/bolivia-carmelo-yujra/!
Processed https://www.coffeereview.com/review/bolivia-familia-miranda/!
Processed https://www.coffeereview.com/review/extra-fancy-kona-coffee-medium-roast/!
Processed https://www.coffeereview.com/review/new-guinea-peaberry-goroka-waghi/!
Processed https://www.coffeereview.com/review/new-guinea-onaka-agoga/!
Processed https://www.coffeereview.com/review/ethiopia-amaro-gayo-natural/!
Processed https://www.coffeereview.com/review/organic-costa-rica-la-amistad-estate/!
Processed https://www.coffeereview.com/review/burundi-ngoma/!
Processed https://www.coffeereview.com/review/paretto-divino/!
Processed https://www.coffeereview.com/review/paretto-celesto/!
Processed https://www.coffeereview.com/review/mon-ami-delight/!
Processed https://www.coffeereview.com/review/mon-ami-bright/!
Processe

Processed https://www.coffeereview.com/review/premio-ditalia-espresso-milano/!
Processed https://www.coffeereview.com/review/espresso-dolci/!
Processed https://www.coffeereview.com/review/kenya-aa-igandene-estate/!
Processed https://www.coffeereview.com/review/kenya-7/!
Processed https://www.coffeereview.com/review/guatemala-el-paraiso/!
Processed https://www.coffeereview.com/review/el-salvador-santa-sofia-peaberry-espresso/!
Processed https://www.coffeereview.com/review/la-minita-peaberry-2/!
Processed https://www.coffeereview.com/review/costa-rica-sombra-del-poro/!
Processed https://www.coffeereview.com/review/don-roberto-espresso-reserve/!
Processed https://www.coffeereview.com/review/el-salvador-santa-sofia-peaberry/!
Processed https://www.coffeereview.com/review/espresso-13/!
Processed https://www.coffeereview.com/review/espresso-blue-label/!
Processed https://www.coffeereview.com/review/single-origin-decaf/!
Processed https://www.coffeereview.com/review/peaberry-medium-roast/!
Pr

Processed https://www.coffeereview.com/review/el-salvador-pacamara-finca-los-alpes-mundani/!
Processed https://www.coffeereview.com/review/165-espresso/!
Processed https://www.coffeereview.com/review/gran-bar-oro-ese-pod/!
Processed https://www.coffeereview.com/review/el-salvador-finca-las-nubes/!
Processed https://www.coffeereview.com/review/kenya-kariaini-estate-microlot-896/!
Processed https://www.coffeereview.com/review/ethiopia-sidamo-natural-special-prep/!
Processed https://www.coffeereview.com/review/nicaraguan-los-placeres-estate/!
Processed https://www.coffeereview.com/review/decaf-colombia-las-serranias/!
Processed https://www.coffeereview.com/review/arbor-day-blend-decaffeinated/!
Processed https://www.coffeereview.com/review/delizia-decaffeinated/!
Processed https://www.coffeereview.com/review/organic-decaf/!
Processed https://www.coffeereview.com/review/media-luna/!
Processed https://www.coffeereview.com/review/decaf-organic-blend/!
Processed https://www.coffeereview.com/r

Processed https://www.coffeereview.com/review/north-bird-espresso/!
Processed https://www.coffeereview.com/review/direct-fair-trade-espresso/!
Processed https://www.coffeereview.com/review/ethiopia-yirgacheffe-17/!
Processed https://www.coffeereview.com/review/costa-rica-los-nacientes-special-reserve/!
Processed https://www.coffeereview.com/review/spring-revival-blend-2/!
Processed https://www.coffeereview.com/review/chachuda-ml/!
Processed https://www.coffeereview.com/review/sweet-heart-blend/!
Processed https://www.coffeereview.com/review/o-sole-mio/!
Processed https://www.coffeereview.com/review/ethiopia-sidama-ardi/!
Processed https://www.coffeereview.com/review/puerto-rican-mountain-grown/!
Processed https://www.coffeereview.com/review/100-hawaiian-coffee-hamakua-estate/!
Processed https://www.coffeereview.com/review/haiti-ranquitte-ecocafe/!
Processed https://www.coffeereview.com/review/jamaica-blue-mountain-4/!
Processed https://www.coffeereview.com/review/kona-extra-fancy-3/!
P

In [18]:
data

[['Colombia Cerro Azul Enano',
  'Equator Coffees',
  'San Rafael, California',
  'Trujillo, Valle del Cauca Department, Colombia',
  'Medium-Light',
  '60/77',
  '$26.00/12 ounces',
  'October 2022',
  '94',
  'Aroma:',
  '9',
  'Acidity/Structure:',
  '9',
  'Body:',
  '9',
  'Flavor:',
  '9',
  'Aftertaste:',
  '8',
  'Blind Assessment: Elegantly fruit- and cocoa-toned. Strawberry guava, cocoa nib, cane sugar, tea rose, cedar in aroma and cup. Delicately sweet structure with gently bright acidity; crisp, syrupy mouthfeel. A resonant finish centers around notes of cocoa nib and tea rose.',
  'Notes: Produced at Finca Cerro Azul (also owners of Granja La Esperanza), entirely of the Enano (dwarf Geisha) variety of Arabica, and processed by the washed method (fruit skin and flesh are removed from the beans before they are dried). Founded in 1995 by Brooke McDonnell and Helen Russell, Equator Coffees operates retail stores in the San Francisco Bay Area and southern California and provide

### Turning List into DataFrame

In [52]:
#creates column headers to use when generating pandas dataframes
cols = ['coffee_name', 'roaster_name', 'roaster_location', 'coffee_origin', 'roast_level', 'agtron', 'est_price', 'review_date', 'overall_score', 'h1', 's1', 'h2', 's2', 'h3', 's3', 'h4', 's4', 'h5', 's5', 'p1', 'p2', 'p3']

In [53]:
data_df = pd.DataFrame(data, columns = cols)

In [55]:
len(data_df)

4779

In [74]:
data_df.to_csv('data_df.csv', index=False)

### Scraping Second Portion of Data

In [21]:
#creates column headers to use when generating pandas dataframes
older_cols = ['coffee_name', 'roaster_name', 'overall_score', 'roaster_location', 'x1', 'coffee_origin', 'x2', 'roast_level', 'x3', 'agtron', 'x4', 'est_price', 'h1', 's1', 'h2', 's2', 'h3', 's3', 'p1', 'p2', 'p3']

In [22]:
older_data = []

In [23]:
#this function scrapes otential data from each review page, leaves off h4, s4 and h5, s5
#those are missing on lots of older pages

def short_scraper(a,b):

    for url in url_list[a:b]:
        req = Request(
            url,  
            headers={'User-Agent': 'Mozilla/5.0'}
)
        page = urlopen(req).read()
        soup = bs(page, 'html.parser')
    
        try: 
            coffee_name = soup.find("h1", class_="review-title").text
            roaster_name = soup.find("p", class_="review-roaster").text
            overall_score = soup.find_all("span")[63].text
            roaster_location = soup.find_all("td")[1].text
            x1 = soup.find_all("td")[2].text
            coffee_origin = soup.find_all('td')[3].text
            x2 = soup.find_all("td")[4].text
            roast_level = soup.find_all('td')[5].text
            x3 = soup.find_all("td")[6].text
            agtron = soup.find_all('td')[7].text
            x4 = soup.find_all("td")[8].text
            est_price = soup.find_all('td')[9].text
            h1 = soup.find_all("td")[10].text
            s1 = soup.find_all("td")[11].text
            h2 = soup.find_all("td")[12].text
            s2 = soup.find_all("td")[13].text
            h3 = soup.find_all("td")[14].text
            s3 = soup.find_all("td")[15].text        
            p1 = soup.find_all("p")[16].text
            p2 = soup.find_all("p")[17].text
            p3 = soup.find_all("p")[18].text
        except IndexError:
            pass
            continue
    
        older_data.append([coffee_name, roaster_name, overall_score, roaster_location, x1, coffee_origin, x2, roast_level, x3, agtron, x4, est_price, h1, s1, h2, s2, h3, s3, p1, p2, p3])
        print(f'Processed {url}!')
        time.sleep(random.randint(1,5))

    print(len(older_data))

In [24]:
short_scraper(5000,6816)

Processed https://www.coffeereview.com/review/doi-chaang-wild-civet-coffee/!
Processed https://www.coffeereview.com/review/kenya-aa-lenana-2/!
Processed https://www.coffeereview.com/review/mele-100-kona-coffee/!
Processed https://www.coffeereview.com/review/kenya-peaberry-thika-gethumbwini/!
Processed https://www.coffeereview.com/review/kenya-gititu-peaberry/!
Processed https://www.coffeereview.com/review/rwanda-ikawa/!
Processed https://www.coffeereview.com/review/rwanda-coopac-cooperative/!
Processed https://www.coffeereview.com/review/burundi/!
Processed https://www.coffeereview.com/review/burundi-bwayi-lot-5/!
Processed https://www.coffeereview.com/review/rwanda-bufcoffee-nyarusiza/!
Processed https://www.coffeereview.com/review/tanzania-hope-peaberry/!
Processed https://www.coffeereview.com/review/rwanda-kabuye-cooperative-full-flavor-roast/!
Processed https://www.coffeereview.com/review/rwanda-coe-lot-16/!
Processed https://www.coffeereview.com/review/rwandan/!
Processed https://

Processed https://www.coffeereview.com/review/full-city-roast-k-cup/!
Processed https://www.coffeereview.com/review/italian-roast-k-cup/!
Processed https://www.coffeereview.com/review/french-roast-k-cup-2/!
Processed https://www.coffeereview.com/review/house-blend-k-cup/!
Processed https://www.coffeereview.com/review/decaffeinated-french-roast-k-cup/!
Processed https://www.coffeereview.com/review/decaffeinated-house-blend-k-cup/!
Processed https://www.coffeereview.com/review/la-crema-espresso/!
Processed https://www.coffeereview.com/review/panama-paso-ancho/!
Processed https://www.coffeereview.com/review/kenya-aa-thunguri-microlot/!
Processed https://www.coffeereview.com/review/hawaiian-holiday-blend/!
Processed https://www.coffeereview.com/review/organic-ethiopian-adado/!
Processed https://www.coffeereview.com/review/ethiopia-yirgacheffe-kochere-2/!
Processed https://www.coffeereview.com/review/guatemala-antigua-la-tacita/!
Processed https://www.coffeereview.com/review/kenya-aa-gachat

Processed https://www.coffeereview.com/review/kenya-peaberry-kaliluni-cooperative/!
Processed https://www.coffeereview.com/review/kenya-wagumuga/!
Processed https://www.coffeereview.com/review/kenya-single-lot-gaturiri-4815/!
Processed https://www.coffeereview.com/review/kenya-fair-trade-kangocho/!
Processed https://www.coffeereview.com/review/kenya-thangathi-aa/!
Processed https://www.coffeereview.com/review/kenya-peaberry-kii/!
Processed https://www.coffeereview.com/review/kenya-aa-gachatha-estate/!
Processed https://www.coffeereview.com/review/kenya-peaberry-nyeri-tatu/!
Processed https://www.coffeereview.com/review/el-salvador-pacamara-tierra-fertil-los-alpes-reserva/!
Processed https://www.coffeereview.com/review/fair-trade-organic-guatemala-maya-ixil-quiche/!
Processed https://www.coffeereview.com/review/kenya-aa-16/!
Processed https://www.coffeereview.com/review/celebes-kalossi-4/!
Processed https://www.coffeereview.com/review/papua-new-guinea-8/!
Processed https://www.coffeerev

Processed https://www.coffeereview.com/review/reserva-del-patron-colombia-narino/!
Processed https://www.coffeereview.com/review/colombia-micro-lot-huila-la-argentina/!
Processed https://www.coffeereview.com/review/colombia-micro-lot-humberto-gonzales/!
Processed https://www.coffeereview.com/review/tres-santos-micro-lot-la-dorada-cauca/!
Processed https://www.coffeereview.com/review/colombia-tolima-jairo-guiterrez-micro-lot-07/!
Processed https://www.coffeereview.com/review/organic-colombia/!
Processed https://www.coffeereview.com/review/costa-rica-dark-roast/!
Processed https://www.coffeereview.com/review/costa-rica-breakfast-blend/!
Processed https://www.coffeereview.com/review/sumatra-mandheling-south-italian-espresso-roast/!
Processed https://www.coffeereview.com/review/nectar/!
Processed https://www.coffeereview.com/review/rwandan-french-roast/!
Processed https://www.coffeereview.com/review/100-arabica/!
Processed https://www.coffeereview.com/review/100-colombia-dark-roast/!
Proce

Processed https://www.coffeereview.com/review/brazil-fazenda-cachoeria-da-grama-cup-of-excellence/!
Processed https://www.coffeereview.com/review/latin-breeze/!
Processed https://www.coffeereview.com/review/el-salvador-santa-barbara-estate-organic/!
Processed https://www.coffeereview.com/review/sumatra-beveo/!
Processed https://www.coffeereview.com/review/amalfi-dark-roast/!
Processed https://www.coffeereview.com/review/espresso-10/!
Processed https://www.coffeereview.com/review/selection-a-light-roast/!
Processed https://www.coffeereview.com/review/supercrema/!
Processed https://www.coffeereview.com/review/gold-espresso/!
Processed https://www.coffeereview.com/review/black-cat-espresso-2/!
Processed https://www.coffeereview.com/review/extra-mild/!
Processed https://www.coffeereview.com/review/garuda-blend/!
Processed https://www.coffeereview.com/review/espresso-coffee/!
Processed https://www.coffeereview.com/review/crema-doro/!
Processed https://www.coffeereview.com/review/premium-bar

...

Processed https://www.coffeereview.com/review/100-sumatra-lintong/!
Processed https://www.coffeereview.com/review/full-city-sumatra-organic-fair-trade/!
Processed https://www.coffeereview.com/review/arabian-mocha-java-2/!
Processed https://www.coffeereview.com/review/casablanca-blend/!
Processed https://www.coffeereview.com/review/dark-sumatra-2/!
Processed https://www.coffeereview.com/review/ethiopia-sidamos/!
Processed https://www.coffeereview.com/review/french-roast-5/!
Processed https://www.coffeereview.com/review/dark-line-red-eye/!
Processed https://www.coffeereview.com/review/costa-rica-4/!
Processed https://www.coffeereview.com/review/italian-roast-3/!
Processed https://www.coffeereview.com/review/ethiopian-yirgacheffe-2/!
Processed https://www.coffeereview.com/review/oromia-dark-roast-fair-trade/!
Processed https://www.coffeereview.com/review/ethiopian-harrar/!
Processed https://www.coffeereview.com/review/best-of-africa-blend-2/!
Processed https://www.coffeereview.com/review/

Processed https://www.coffeereview.com/review/songbird-shade-grown-costa-rican/!
Processed https://www.coffeereview.com/review/sumatra-french-roast/!
Processed https://www.coffeereview.com/review/organic-sumatran-reserve-3/!
Processed https://www.coffeereview.com/review/organic-mexican-select-2/!
Processed https://www.coffeereview.com/review/guatemala-atitlan-la-voz/!
Processed https://www.coffeereview.com/review/organic-peru-la-florida/!
Processed https://www.coffeereview.com/review/organic-guatemala-highlands/!
Processed https://www.coffeereview.com/review/nicaragua-miraflor/!
Processed https://www.coffeereview.com/review/beethovens-viennese-blend/!
Processed https://www.coffeereview.com/review/fair-trade-coffee-decaf/!
Processed https://www.coffeereview.com/review/daybreak-morning-blend/!
Processed https://www.coffeereview.com/review/chuck-roast/!
Processed https://www.coffeereview.com/review/pride-of-the-andes/!
Processed https://www.coffeereview.com/review/solstice-blend/!
Process

Processed https://www.coffeereview.com/review/ethiopian-harrar-horse/!
Processed https://www.coffeereview.com/review/ethiopia-washed-sidamo-vienna-roast/!
Processed https://www.coffeereview.com/review/organic-timor-aifu-vienna-roast/!
Processed https://www.coffeereview.com/review/paia-blend/!
Processed https://www.coffeereview.com/review/ethiopia-harrar/!
Processed https://www.coffeereview.com/review/ethiopia-yirgacheffe-6/!
Processed https://www.coffeereview.com/review/mocca-java-blend/!
Processed https://www.coffeereview.com/review/mexican-maya/!
Processed https://www.coffeereview.com/review/oakhurst-morning/!
Processed https://www.coffeereview.com/review/ethiopian-yirgacheffe/!
Processed https://www.coffeereview.com/review/don-pedro-supremo/!
Processed https://www.coffeereview.com/review/caf-virgen-de-oro/!
Processed https://www.coffeereview.com/review/tuluni-supremo/!
Processed https://www.coffeereview.com/review/guatemalan-antigua-1999-2/!
Processed https://www.coffeereview.com/re

Processed https://www.coffeereview.com/review/costa-rican-viennese/!
Processed https://www.coffeereview.com/review/vienna-roast/!
Processed https://www.coffeereview.com/review/brazil-fazenda-vista-alegre/!
Processed https://www.coffeereview.com/review/guatemala-finca-bella-carmona/!
Processed https://www.coffeereview.com/review/black-cat-blend/!
Processed https://www.coffeereview.com/review/elm-city-house-blend/!
Processed https://www.coffeereview.com/review/guatemala-la-flor-del-cafe/!
Processed https://www.coffeereview.com/review/kenya-aa-7/!
Processed https://www.coffeereview.com/review/kenya-nyaithee-aa/!
Processed https://www.coffeereview.com/review/kenya-aa-8/!
Processed https://www.coffeereview.com/review/bayview-farms-kona-extra-fancy/!
Processed https://www.coffeereview.com/review/mocha-java-blend-2/!
Processed https://www.coffeereview.com/review/yemen-mocha-mattari/!
Processed https://www.coffeereview.com/review/guatemala-coban/!
Processed https://www.coffeereview.com/review/

Processed https://www.coffeereview.com/review/espresso-dolce/!
Processed https://www.coffeereview.com/review/major-dickasons-blend-2/!
Processed https://www.coffeereview.com/review/espresso-blend/!
Processed https://www.coffeereview.com/review/illy-espresso/!
Processed https://www.coffeereview.com/review/vesuvio-blend/!
Processed https://www.coffeereview.com/review/napoli-blend/!
Processed https://www.coffeereview.com/review/perugia-blend/!
Processed https://www.coffeereview.com/review/el-salvador-green-8/!
Processed https://www.coffeereview.com/review/el-salvador-green-7/!
Processed https://www.coffeereview.com/review/el-salvador-green-6/!
Processed https://www.coffeereview.com/review/el-salvador-green-5/!
Processed https://www.coffeereview.com/review/el-salvador-green-4/!
Processed https://www.coffeereview.com/review/el-salvador-green-3/!
Processed https://www.coffeereview.com/review/pacamara-el-98-green/!
Processed https://www.coffeereview.com/review/bourbon-green/!
Processed https:

### Turning Second Portion into DataFrame

In [25]:
older_data_df = pd.DataFrame(older_data, columns = older_cols)

In [26]:
older_data_df.head()

,coffee_name,roaster_name,overall_score,roaster_location,x1,coffee_origin,x2,roast_level,x3,agtron,...,est_price,h1,s1,h2,s2,h3,s3,p1,p2,p3
0,Doi Chaang Wild Civet Coffee,Doi Chaang Coffee,90,"Calgary, Alberta, Canada",Coffee Origin:,Northern Thailand.,Roast Level:,Medium-Light,Agtron:,49/80,...,June 2009,Aroma:,8,Acidity:,8,Body:,7\t\t\t\t\t\t\t,Blind Assessment: Intriguing mid-tones through...,Notes: Doi Chaang is a single-estate coffee pr...,Who Should Drink It: Culinary adventurers who ...
1,Kenya AA Lenana,Après Coffee,92,"Lancaster, Pennsylvania",Coffee Origin:,South-central Kenya,Roast Level:,Medium-Dark,Agtron:,42/53,...,June 2009,Aroma:,8,Acidity:,8,Body:,7\t\t\t\t\t\t\t,"Blind Assessment: Rich, very intense aroma: da...",Notes: Despite stresses brought on by social u...,Who Should Drink It: Those who prefer understa...
2,Mele 100% Kona Coffee,Hula Daddy,92,"Holualoa, Hawaii",Coffee Origin:,"Holualoa, North Kona growing district, Hawaii.",Roast Level:,Medium-Light,Agtron:,51/73,...,June 2009,Aroma:,8,Acidity:,8,Body:,7\t\t\t\t\t\t\t,"Blind Assessment: An exciting, rather unusual ...",Notes: A blend of coffees processed by a varie...,Who Should Drink It: An exhilarating sensory r...
3,Kenya Peaberry Thika Gethumbwini,JBC Coffee Roasters,96,"Madison, Wisconsin",Coffee Origin:,South-central Kenya,Roast Level:,Light,Agtron:,57/90,...,June 2009,Aroma:,8,Acidity:,9,Body:,8\t\t\t\t\t\t\t,"Blind Assessment: Clean, complex, impeccable. ...",Notes: Despite national coffee leadership mark...,Who Should Drink It: Strikingly complete expre...
4,Kenya Gititu Peaberry,Atomic Cafe Coffee Roasters,92,"Beverly, Massachusetts",Coffee Origin:,South-central Kenya,Roast Level:,Medium,Agtron:,44/63,...,June 2009,Aroma:,8,Acidity:,8,Body:,8\t\t\t\t\t\t\t,"Blind Assessment: Rich, complex fruit aroma, i...",Notes: Despite stresses brought on by social u...,Who Should Drink It: Lovers of sweet fruit fla...


In [27]:
len(older_data)

1788

In [28]:
older_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1788 entries, 0 to 1787
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   coffee_name       1788 non-null   object
 1   roaster_name      1788 non-null   object
 2   overall_score     1788 non-null   object
 3   roaster_location  1788 non-null   object
 4   x1                1788 non-null   object
 5   coffee_origin     1788 non-null   object
 6   x2                1788 non-null   object
 7   roast_level       1788 non-null   object
 8   x3                1788 non-null   object
 9   agtron            1788 non-null   object
 10  x4                1788 non-null   object
 11  est_price         1788 non-null   object
 12  h1                1788 non-null   object
 13  s1                1788 non-null   object
 14  h2                1788 non-null   object
 15  s2                1788 non-null   object
 16  h3                1788 non-null   object
 17  s3            

In [29]:
older_data_df.to_csv('older_data_df.csv', index=False)

## See "Capstone Project Part 2" Notebook for Next Steps